In [1]:
import time
from detector import FaceDetector
import cv2
import av
from IPython.display import Video

In [9]:
# Функция для отображения FPS на кадре с измененным положением текста
def draw_fps(frame, fps):
    font_scale = 2.5
    font_thickness = 4
    text = f"FPS: {int(fps)}"
    # Получение размеров текста
    (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
    # Положение текста в левом верхнем углу
    position = (10, text_height + 10)  # Смещаем текст вниз на высоту текста плюс некоторый отступ
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 0), font_thickness)

In [ ]:
face_detector = FaceDetector(model_path='best.pt',
                             device='0'
#                              crops_path="crops/",
#                              meta_path="meta/meta.json",
#                              marks_path="marks/marks.json"
                            )

input_filename = "assets/train_station.mp4"
output_filename = "inference_train_station.mp4"

input_container = av.open(input_filename)
output_container = av.open(output_filename, "w")
stream = output_container.add_stream("h264")

frame_count = 0
fps = 0
# Запись изображений в видео
for frame in input_container.decode(video=0):
    start_time = time.time()
    if frame_count > 500:
        break
    # Получаем изображение кадра
    image = frame.to_ndarray(format='rgb24')

    # Обрабатываем кадр с помощью детектора лиц
    results = face_detector.update(image)
    annotated_frame = results[0].plot()
    draw_fps(annotated_frame, fps)
    
    frame = av.VideoFrame.from_ndarray(annotated_frame, format="rgb24")
    packet = stream.encode(frame)
    output_container.mux(packet)
    
    fps = 1 / (time.time() - start_time)
    print("fps: ", fps)
    frame_count += 1
        
# Завершение записи видео
packet = stream.encode(None)  # Посылаем None, чтобы завершить видеопоток
output_container.mux(packet)
# Закрытие видеопотока
output_container.close()
input_container.close()

print(f"Видео сгенерировано: {output_filename}")

Ultralytics YOLOv8.1.27 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)
Setup complete ✅ (80 CPUs, 754.5 GB RAM, 4083.0/4434.8 GB disk)

0: 384x640 4 faces, 7.6ms
Speed: 6.0ms preprocess, 7.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
fps:  6.505136731351432

0: 384x640 4 faces, 6.0ms
Speed: 2.3ms preprocess, 6.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
fps:  34.928083674761

0: 384x640 4 faces, 6.1ms
Speed: 2.3ms preprocess, 6.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
fps:  33.839495913575966

0: 384x640 4 faces, 5.9ms
Speed: 2.7ms preprocess, 5.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
fps:  34.64391380122079

0: 384x640 4 faces, 5.8ms
Speed: 2.4ms preprocess, 5.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
fps:  38.08260620863107

0: 384x640 4 faces, 6.8ms
Speed: 2.3ms preprocess, 6.8ms inference, 1.2ms postprocess per image at sha

In [12]:
Video("inference_train_station.mp4", width=990, height=360)